In [22]:
import pandas as pd
import numpy as np
# 读取index_daily_auction.parquet文件
df = pd.read_parquet('../data/index_daily_auction.parquet')

#使用ta-lib 计算 money、volume 的5 、10 、20、30、60日均线，形成指标，
import talib

# 计算index_code，分组计算5日均线，不包含当日的，从T-6到T-1日
df['MA5'] = df.groupby('index_code')['money'].transform(lambda x: talib.SMA(x, timeperiod=5).shift(1))
df['ma5_ratio'] = df['money'] / df['MA5']

df['MA10'] = df.groupby('index_code')['money'].transform(lambda x: talib.SMA(x, timeperiod=10).shift(1))
df['ma10_ratio'] = df['money'] / df['MA10']

df['MA60'] = df.groupby('index_code')['money'].transform(lambda x: talib.SMA(x, timeperiod=60).shift(1))
df['ma60_ratio'] = df['money'] / df['MA60']


# 使用下一日的current和当日的current的值，计算1d 、3d、5d 收益率
df['return_1d'] = df['current'].pct_change()
df['return_3d'] = df['current'].pct_change(periods=3)
df['return_5d'] = df['current'].pct_change(periods=5)


#  ma5上穿ma10时，为1，下穿为-1，其他为0
df['ma5_signal'] = np.where(df['ma5_ratio'] > df['ma10_ratio'], 1, np.where(df['ma5_ratio'] < df['ma10_ratio'], -1, 0))

# 当日money 除以 ma5 ，得到指标
df['current_signal'] = df['money'] / df['MA5']


df.head()

,index_code,date,current,volume,money,MA5,ma5_ratio,MA10,ma10_ratio,MA60,ma60_ratio,return_1d,return_3d,return_5d,ma5_signal,current_signal
0,399003.XSHE,2024-01-02 09:25:00,7404.10,915055.0,4.248600e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
1,399005.XSHE,2024-01-02 09:25:00,6032.45,30671711.0,5.015447e+08,NaN,NaN,NaN,NaN,NaN,NaN,-0.185255,NaN,NaN,0,NaN
2,399008.XSHE,2024-01-02 09:25:00,1221.87,53455953.0,7.310460e+08,NaN,NaN,NaN,NaN,NaN,NaN,-0.797450,NaN,NaN,0,NaN
3,399013.XSHE,2024-01-02 09:25:00,4287.41,70911155.0,1.098297e+09,NaN,NaN,NaN,NaN,NaN,NaN,2.508892,-0.420941,NaN,0,NaN
4,399017.XSHE,2024-01-02 09:25:00,3329.44,21881737.0,4.208399e+08,NaN,NaN,NaN,NaN,NaN,NaN,-0.223438,-0.448078,NaN,0,NaN


In [24]:
# 读取D:\workspace\xiaoyao\data\index_industry.csv，形成df
df_industry = pd.read_csv(r'D:\workspace\xiaoyao\data\index_industry.csv',encoding='utf-8')
df_industry.head()


df = df[df['ma60_ratio']>5]
top_10 = df.groupby('date').apply(lambda x: x.nlargest(20, 'ma60_ratio')).reset_index(drop=True)

# 合并top_10 和 df_industry，用index_code 和 指数代码 列  并将概念和行业信息添加到top_10。
top_10 = pd.merge(top_10,df_industry[['index_code','指数名称']],on='index_code',how='right')

# top_10 按日期、index_code排序
top_10 = top_10.sort_values(['date'])

# 按日分组，再按current_signal排序，获取每个日期的前10个指数
top_10.to_csv('index_top10.csv')

C:\Users\jay\AppData\Local\Temp\ipykernel_41408\1998019001.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_10 = df.groupby('date').apply(lambda x: x.nlargest(20, 'ma60_ratio')).reset_index(drop=True)
